In [67]:
import h5py 
import json
import numpy as np

# load data from hdf5 files
datasets = ["beatles","rw","rwc_pop","queen"]
source_separation = ["vocals","drums","both","none"]
results = {}
for ii in range(4):
    dataset_results = {}
    for jj in range(4):
        with h5py.File(f"../results/majmin_rp_25_50/{source_separation[jj]}.hdf5", 'r') as file:
            # load parameters
            params = json.loads(file['/parameters'][()])
            # print(params)
            # load majmin scores
            scores = {}
            for track_id in file[f"/{datasets[ii]}"]:
                track_data = file[f"/{datasets[ii]}/{track_id}"]

                # harmonic mean, seg_score
                majmin_score,seg_score = track_data.attrs.get("majmin"),track_data.attrs.get("seg")
                f_score = (2 * majmin_score * seg_score) / (majmin_score + seg_score)
                scores[track_id] = f_score
        dataset_results[source_separation[jj]] = scores
    results[datasets[ii]] = dataset_results

# violin plot
f_score = []
for dataset_name in datasets:
    for track_id in results[dataset_name]["drums"]:
        f_score.append(results[dataset_name]["drums"][track_id])
print(np.mean(np.array(f_score)))

## paired T-Test ##
source_sep = "both"
delta = []
for dataset_name in datasets:
    for track_id in results[dataset_name]["none"]:
        delta.append(results[dataset_name][source_sep][track_id] - results[dataset_name]["none"][track_id] )
N = len(delta)
delta = np.array(delta)
delta_mu = np.mean(delta)
delta_std = np.std(delta,ddof=1) / np.sqrt(N)
t_obs = delta_mu/(delta_std)
print(f"t_obs={t_obs}")

0.7517920931387099
t_obs=5.921248632506252


Dataset statistics
and creation of training data

In [12]:
import matplotlib.pyplot as plt
from dataloader import Dataloader
import utils
import features

dataset = Dataloader("beatles","/home/max/ET-TI/Masterarbeit/chord-recognition/mirdata/",None)

#### chord recognition experiment #### 
fpath = "/home/max/ET-TI/Masterarbeit/training_data.hdf5"
file = h5py.File(fpath, 'w')
print(f"created file {fpath}")
group_splits = file.create_group("/splits")

# metadata collection
t_audio = 0.0
chords = {}
for split in range(9):
    subgroup = group_splits.create_group(f"{split}")
    for track_id in dataset.getExperimentSplits(split):
        
        audiopath,annotationpath = dataset[track_id]
        y = utils.loadAudiofile(audiopath)
        intervals,labels = utils.loadChordAnnotations(annotationpath)
        
        # accumulate total chord duration
        for interval,label in zip(intervals,labels):
            duration = chords.get(label,0.0)
            duration += (interval[1]-interval[0])
            chords.update({label:duration})
    
        # compute chromagram
        chroma_params = {"nCRP":33,"hop_length":2048,"fs":22050}
        chroma = features.crpChroma(y,nCRP=33,liftering=True,window=False)
        t_chroma = utils.timeVector(chroma.shape[1],hop_length=2048)
        

0111
N 0.0
E:min 0.0
A:min 0.0
E:min 3.006175
G 0.0
F 0.0
B 0.0
E 0.0
G#:min 0.0
G:min 0.0
F#:min 0.0
B 3.6762309999999996
E 0.9984590000000004
G#:min 0.5108389999999989
G:min 0.4760080000000002
F#:min 0.9171890000000005
B 4.651469999999998
E 1.9620870000000004
G#:min 1.0216779999999996
G:min 0.9752369999999999
F#:min 1.8808170000000004
C:min6 0.0
E 2.914104000000002
G#:min 1.4976879999999984
G:min 1.4512450000000001
F#:min 2.867666
B:7 0.0
E 3.8661210000000032
G#:min 1.996916999999998
G:min 1.9388639999999988
F#:min 3.8429040000000008
B 5.615097999999998
A 0.0
B 6.578725999999998
C#:min 0.0
F#:min 4.829752000000003
B 8.471151999999998
E 4.806529000000005
G#:min 2.4729269999999968
G 2.995374
F#:min 5.863040000000009
B 9.411559999999996
E 5.758547000000007
G#:min 2.9837669999999914
G:min 2.4264819999999965
F#:min 6.803449000000004
B 10.375187999999996
E 6.675734000000006
G#:min 3.4713859999999936
G:min 2.914101999999996
F#:min 7.778686000000004
C:min6 0.9171890000000005
E 7.685801000000

KeyboardInterrupt: 